### 2. Azure Document Intelligence 기반 역사문서 OCR 분석 및 PIL 시각화
- 주요 역할: Azure Document Intelligence OCR 결과를 분석하여 세로줄별로 그룹화하고 시각화
- 핵심 라이브러리: PIL(Pillow), Azure Document Intelligence, matplotlib
- 특징: 한글 경로 지원, 세로줄 기반 텍스트 분석, 신뢰도 통계 제공

In [1]:
pip install opencv-python pillow matplotlib numpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install azure-ai-documentintelligence

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
"""
Azure Document Intelligence OCR 완전 분석 도구
- 실시간 Azure API 호출 및 OCR 분석
- 한자/고문서 시각화 지원
- 설정 파일 자동 생성
- 환경변수 기반 보안 설정
"""

import os
import json
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.font_manager as fm
from typing import List, Tuple, Dict, Optional
import platform
import warnings
from pathlib import Path
from datetime import datetime

# 환경변수 로드
try:
    from dotenv import load_dotenv
    load_dotenv()
    print("✅ .env 파일 로드 완료")
except ImportError:
    print("⚠️ python-dotenv 설치 필요: pip install python-dotenv")

# Azure SDK imports
try:
    from azure.core.credentials import AzureKeyCredential
    from azure.ai.documentintelligence import DocumentIntelligenceClient
    from azure.ai.documentintelligence.models import AnalyzeDocumentRequest
    AZURE_SDK_AVAILABLE = True
    print("✅ Azure SDK 로드 완료")
except ImportError:
    AZURE_SDK_AVAILABLE = False
    print("❌ Azure SDK 없음. 설치 필요: pip install azure-ai-documentintelligence")

# 경고 메시지 숨기기
warnings.filterwarnings('ignore', message='Glyph.*missing from font')


class CompleteAzureOCRAnalyzer:
    """완전한 Azure Document Intelligence OCR 분석기"""
    
    def __init__(self, config_path: Optional[str] = None):
        """
        초기화
        Args:
            config_path: Azure 설정 파일 경로 (선택사항)
        """
        print("🚀 Azure OCR 분석기 초기화 중...")
        
        # 색상 정의
        self.colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown', 
                      'pink', 'gray', 'olive', 'cyan']
        
        # Azure 설정 로드
        self.config = self._load_azure_config(config_path)
        self.document_client = None
        
        # 폰트 설정
        self.han_font = self._setup_cjk_font()
        self._configure_matplotlib()
        
        print(f"✅ 초기화 완료 - 폰트: {self._get_font_name()}")
    
    def _load_azure_config(self, config_path: Optional[str]) -> Dict:
        """Azure 설정 로드 (환경변수 우선 > 설정파일)"""
        config = {}
        
        # 1. 환경변수 우선 확인 (보안상 권장)
        env_endpoint = os.getenv('AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT')
        env_key = os.getenv('AZURE_DOCUMENT_INTELLIGENCE_KEY')
        
        if env_endpoint and env_key:
            config['endpoint'] = env_endpoint
            config['key'] = env_key
            print("✅ 환경변수(.env)에서 Azure 설정 로드")
            return config
        
        print("⚠️ 환경변수에 Azure 설정이 없습니다. 설정 파일을 확인합니다...")
        
        # 2. 설정 파일에서 로드 (환경변수가 없을 때만)
        config_files = [
            config_path,
            'azure_config.json',
            'config.json',
            'azure_config_template.json'  # 템플릿 파일은 마지막에
        ]
        
        for file_path in config_files:
            if file_path and os.path.exists(file_path):
                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        file_config = json.load(f)
                        endpoint = file_config.get('endpoint', '')
                        key = file_config.get('key', '')
                        
                        # 실제 Azure 설정인지 확인 (템플릿 값이 아닌지)
                        if (endpoint and key and 
                            not endpoint.startswith('https://your-resource') and
                            not key.startswith('your-32-character')):
                            config.update(file_config)
                            print(f"✅ 설정 파일에서 Azure 설정 로드: {file_path}")
                            return config
                        else:
                            print(f"⚠️ 템플릿 설정 파일 건너뜀: {file_path}")
                            
                except Exception as e:
                    print(f"⚠️ 설정 파일 읽기 실패: {file_path} - {e}")
        
        print("❌ Azure 설정을 찾을 수 없습니다.")
        print("💡 .env 파일에 AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT와 AZURE_DOCUMENT_INTELLIGENCE_KEY를 설정하세요.")
        return config
    
    def setup_azure_client(self) -> bool:
        """Azure Document Intelligence 클라이언트 설정"""
        if not AZURE_SDK_AVAILABLE:
            print("❌ Azure SDK 설치 필요:")
            print("pip install azure-ai-documentintelligence")
            return False
        
        endpoint = self.config.get('endpoint')
        key = self.config.get('key')
        
        if not endpoint or not key:
            print("❌ Azure 설정이 누락되었습니다.")
            self._show_azure_setup_guide()
            return False
        
        try:
            self.document_client = DocumentIntelligenceClient(
                endpoint=endpoint,
                credential=AzureKeyCredential(key)
            )
            print("✅ Azure Document Intelligence 클라이언트 연결 성공")
            return True
        except Exception as e:
            print(f"❌ Azure 클라이언트 연결 실패: {e}")
            return False
    
    def _show_azure_setup_guide(self):
        """Azure 설정 가이드 출력"""
        print("\n📝 Azure Document Intelligence 설정 가이드:")
        print("=" * 60)
        print("권장 방법 - .env 파일 사용:")
        print("1. 프로젝트 루트에 .env 파일 생성")
        print("2. 다음 내용 추가:")
        print("   AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT=https://your-resource.cognitiveservices.azure.com/")
        print("   AZURE_DOCUMENT_INTELLIGENCE_KEY=your-api-key-here")
        print("\n대안 방법 - config.json 파일:")
        print('{"endpoint": "https://your-resource.cognitiveservices.azure.com/", "key": "your-api-key"}')
        print("\n자동 템플릿 생성:")
        print("create_azure_config() 함수를 실행하세요.")
        print("=" * 60)
    
    # ...existing code... (analyze_document_live 메서드는 그대로 유지)
    
    def analyze_document_live(self, image_path: str) -> Optional[Dict]:
        """실시간 Azure Document Intelligence 분석"""
        if not self.setup_azure_client():
            return None
        
        if not os.path.exists(image_path):
            print(f"❌ 이미지 파일이 없습니다: {image_path}")
            return None
        
        try:
            print(f"📄 Azure로 문서 분석 중: {os.path.basename(image_path)}")
            print("⏳ 분석 진행 중... (30초-2분 소요)")
            
            # 이미지 파일 읽기
            with open(image_path, 'rb') as f:
                image_data = f.read()
            
            # Azure Document Intelligence API 호출
            poller = self.document_client.begin_analyze_document(
                "prebuilt-read",
                AnalyzeDocumentRequest(bytes_source=image_data)
            )
            
            # 결과 대기
            result = poller.result()
            
            # 표준 JSON 형태로 변환
            ocr_result = {
                "analyzeResult": {
                    "pages": [],
                    "version": "4.0",
                    "apiVersion": "2024-02-29-preview",
                    "modelId": "prebuilt-read",
                    "stringIndexType": "textElements"
                }
            }
            
            total_words = 0
            total_confidence = 0
            
            for page_idx, page in enumerate(result.pages):
                page_data = {
                    "pageNumber": page_idx + 1,
                    "words": []
                }
                
                if hasattr(page, 'words') and page.words:
                    for word in page.words:
                        word_data = {
                            "content": word.content,
                            "confidence": word.confidence if hasattr(word, 'confidence') else 0.9,
                            "polygon": list(word.polygon) if hasattr(word, 'polygon') else [],
                            "span": {
                                "offset": word.span.offset if hasattr(word, 'span') else 0,
                                "length": word.span.length if hasattr(word, 'span') else len(word.content)
                            }
                        }
                        page_data["words"].append(word_data)
                        total_words += 1
                        total_confidence += word_data["confidence"]
                
                ocr_result["analyzeResult"]["pages"].append(page_data)
            
            # 분석 결과 통계
            avg_confidence = total_confidence / total_words if total_words > 0 else 0
            print(f"✅ Azure 분석 완료:")
            print(f"   📊 총 {total_words}개 단어 인식")
            print(f"   📊 평균 신뢰도: {avg_confidence:.3f}")
            
            # 결과 자동 저장
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            auto_save_path = f"{Path(image_path).stem}_azure_ocr_{timestamp}.json"
            self._save_ocr_result(ocr_result, auto_save_path)
            
            return ocr_result
            
        except Exception as e:
            print(f"❌ Azure 분석 실패: {e}")
            return None
    
    # ...existing code... (나머지 메서드들은 그대로 유지)
    
    def _save_ocr_result(self, ocr_data: Dict, file_path: str):
        """OCR 결과 저장"""
        try:
            with open(file_path, 'w', encoding='utf-8') as f:
                json.dump(ocr_data, f, ensure_ascii=False, indent=2)
            print(f"💾 OCR 결과 저장: {file_path}")
        except Exception as e:
            print(f"⚠️ 결과 저장 실패: {e}")
    
    def _get_font_name(self) -> str:
        """폰트 이름 안전하게 추출"""
        try:
            if hasattr(self.han_font, 'get_name'):
                return self.han_font.get_name()
            elif hasattr(self.han_font, '_family'):
                family = self.han_font._family
                return family[0] if isinstance(family, list) else str(family)
            else:
                return "시스템 기본 폰트"
        except:
            return "폰트 정보 확인 불가"
    
    def _setup_cjk_font(self):
        """CJK(한중일) 폰트 설정"""
        system = platform.system()
        
        # Windows 폰트 우선순위
        if system == "Windows":
            font_candidates = [
                ("C:/Windows/Fonts/simsun.ttc", "SimSun"),      # 중국어 - 한자 완전 지원
                ("C:/Windows/Fonts/msyh.ttc", "Microsoft YaHei"), # 중국어
                ("C:/Windows/Fonts/batang.ttc", "Batang"),       # 한국어
                ("C:/Windows/Fonts/gulim.ttc", "Gulim"),         # 한국어
                ("C:/Windows/Fonts/malgun.ttf", "Malgun Gothic"), # 한국어
            ]
            
            # 파일 경로로 직접 로드
            for font_path, font_name in font_candidates:
                if os.path.exists(font_path):
                    try:
                        return fm.FontProperties(fname=font_path)
                    except:
                        continue
            
            # 시스템 폰트 이름으로 검색
            system_fonts = ["SimSun", "Microsoft YaHei", "Batang", "Malgun Gothic"]
            available_fonts = {f.name for f in fm.fontManager.ttflist}
            
            for font_name in system_fonts:
                if font_name in available_fonts:
                    try:
                        return fm.FontProperties(family=font_name)
                    except:
                        continue
        
        # macOS
        elif system == "Darwin":
            macos_fonts = ["PingFang SC", "Hiragino Sans GB", "STSong"]
            available_fonts = {f.name for f in fm.fontManager.ttflist}
            
            for font_name in macos_fonts:
                if font_name in available_fonts:
                    try:
                        return fm.FontProperties(family=font_name)
                    except:
                        continue
        
        # Linux
        else:
            linux_fonts = ["Noto Sans CJK SC", "Source Han Sans", "WenQuanYi Micro Hei"]
            available_fonts = {f.name for f in fm.fontManager.ttflist}
            
            for font_name in linux_fonts:
                if font_name in available_fonts:
                    try:
                        return fm.FontProperties(family=font_name)
                    except:
                        continue
        
        return fm.FontProperties()  # 기본 폰트
    
    def _configure_matplotlib(self):
        """matplotlib 전역 설정"""
        try:
            font_name = self._get_font_name()
            if font_name not in ["시스템 기본 폰트", "폰트 정보 확인 불가"]:
                plt.rcParams['font.family'] = [font_name, 'sans-serif']
            
            plt.rcParams['axes.unicode_minus'] = False
            plt.rcParams['font.size'] = 12
            
        except Exception:
            plt.rcParams['font.family'] = ['sans-serif']
            plt.rcParams['axes.unicode_minus'] = False
    
    def group_words_by_vertical_lines(self, words_data: List[Dict], threshold: float = 50) -> List[List[Dict]]:
        """단어들을 세로줄로 그룹화"""
        if not words_data:
            return []
        
        sorted_words = sorted(words_data, key=lambda w: w['polygon'][0])
        vertical_lines = []
        current_line = [sorted_words[0]]
        
        for word in sorted_words[1:]:
            current_line_avg_x = sum(w['polygon'][0] for w in current_line) / len(current_line)
            current_word_x = word['polygon'][0]
            
            if abs(current_word_x - current_line_avg_x) <= threshold:
                current_line.append(word)
            else:
                current_line.sort(key=lambda w: w['polygon'][1])
                vertical_lines.append(current_line)
                current_line = [word]
        
        if current_line:
            current_line.sort(key=lambda w: w['polygon'][1])
            vertical_lines.append(current_line)
        
        return vertical_lines
    
    def create_dual_visualization(self, image_path: str, ocr_data: Dict, save_path: Optional[str] = None):
        """좌우 구성 시각화 생성"""
        print(f"🎨 시각화 생성: {os.path.basename(image_path)}")
        
        # 단어 데이터 추출
        words_data = []
        for page in ocr_data["analyzeResult"]["pages"]:
            words_data.extend(page["words"])
        
        if not words_data:
            print("❌ OCR 데이터가 비어있습니다.")
            return None
        
        print(f"📊 총 {len(words_data)}개 단어 추출")
        
        # 세로줄 그룹화
        vertical_lines = self.group_words_by_vertical_lines(words_data)
        print(f"📊 {len(vertical_lines)}개 세로줄로 그룹화")
        
        # 원본 이미지 로드
        try:
            original_image = Image.open(image_path)
            width, height = original_image.size
        except Exception as e:
            print(f"❌ 이미지 로드 실패: {e}")
            return None
        
        # 좌측: 색상 박스 이미지 생성
        boxed_image = self._create_colored_boxes(image_path, vertical_lines)
        
        # 우측: 통계 및 블럭 생성
        line_stats = self._create_line_statistics(vertical_lines)
        
        # matplotlib 시각화
        fig, axes = plt.subplots(1, 2, figsize=(24, 16))
        
        # 좌측 패널
        axes[0].imshow(boxed_image)
        axes[0].set_title('세로줄별 색상 박스 시각화', fontproperties=self.han_font, fontsize=8)
        axes[0].axis('off')
        
        # 우측 패널
        white_image = np.ones((height, width, 3), dtype=np.uint8) * 255
        axes[1].imshow(white_image)
        axes[1].set_title('한자 지원 텍스트 블럭 방식', fontproperties=self.han_font, fontsize=8)
        axes[1].axis('off')
        
        # 우측 패널에 텍스트 블럭 추가
        rendered_count = 0
        total_count = 0
        
        for idx, (line_name, stats) in enumerate(line_stats.items()):
            x, y, w, h = stats['bbox']
            color = self.colors[idx % len(self.colors)]
            
            # 테두리 추가
            rect = patches.Rectangle((x, y), w, h, linewidth=4, edgecolor=color, facecolor='none')
            axes[1].add_patch(rect)
            
            # 텍스트 렌더링
            for word in stats['words']:
                total_count += 1
                wx, wy, ww, wh = word['bbox']
                text = word['text']
                
                try:
                    axes[1].text(wx + ww/2, wy + wh/2, text,
                               fontproperties=self.han_font,
                               fontsize=18, color='black',
                               verticalalignment='center', horizontalalignment='center',
                               fontweight='bold')
                    rendered_count += 1
                except Exception:
                    # 폰트 문제 시 기본 렌더링
                    try:
                        axes[1].text(wx + ww/2, wy + wh/2, text,
                                   fontsize=18, color='black',
                                   verticalalignment='center', horizontalalignment='center')
                        rendered_count += 1
                    except:
                        pass
        
        # 전체 제목
        avg_confidence = np.mean([stats['avg_confidence'] for stats in line_stats.values()])
        render_rate = (rendered_count / total_count * 100) if total_count > 0 else 0
        
        title = f'Azure Document Intelligence OCR 완전 분석 결과\n평균 신뢰도: {avg_confidence:.3f} | 렌더링: {render_rate:.1f}%'
        
        try:
            fig.suptitle(title, fontproperties=self.han_font, fontsize=18, y=0.95)
        except:
            fig.suptitle(title, fontsize=18, y=0.95)
        
        plt.tight_layout()
        
        # 저장 또는 표시
        if save_path:
            plt.savefig(save_path, dpi=300, bbox_inches='tight')
            print(f"💾 시각화 저장: {save_path}")
        else:
            plt.show()
        
        # 상세 통계 출력
        print(f"\n📊 상세 분석 결과:")
        print(f"   🎯 텍스트 렌더링: {rendered_count}/{total_count} ({render_rate:.1f}%)")
        print(f"   📏 평균 신뢰도: {avg_confidence:.3f}")
        print(f"   📄 세로줄 수: {len(vertical_lines)}")
        
        for line_name, stats in line_stats.items():
            print(f"   {line_name}: {stats['word_count']}개 단어, "
                  f"신뢰도 {stats['avg_confidence']:.3f}, "
                  f"텍스트: {stats['full_text'][:20]}...")
        
        return line_stats
    
    def _create_colored_boxes(self, image_path: str, vertical_lines: List[List[Dict]]) -> np.ndarray:
        """색상 박스 이미지 생성"""
        try:
            pil_image = Image.open(image_path)
            if pil_image.mode != 'RGB':
                pil_image = pil_image.convert('RGB')
            
            draw = ImageDraw.Draw(pil_image)
            
            color_map = {
                'red': (255, 0, 0), 'blue': (0, 0, 255), 'green': (0, 255, 0),
                'orange': (255, 165, 0), 'purple': (128, 0, 128), 'brown': (165, 42, 42),
                'pink': (255, 192, 203), 'gray': (128, 128, 128), 'olive': (128, 128, 0),
                'cyan': (0, 255, 255)
            }
            
            for line_idx, line_words in enumerate(vertical_lines):
                color_name = self.colors[line_idx % len(self.colors)]
                color = color_map.get(color_name, (255, 0, 0))
                
                for word in line_words:
                    polygon = word['polygon']
                    points = [(polygon[i], polygon[i+1]) for i in range(0, len(polygon), 2)]
                    
                    # 테두리
                    draw.polygon(points, outline=color, width=3)
                    
                    # 반투명 채우기
                    overlay = Image.new('RGBA', pil_image.size, (0, 0, 0, 0))
                    overlay_draw = ImageDraw.Draw(overlay)
                    overlay_draw.polygon(points, fill=(*color, 77))
                    
                    pil_image = Image.alpha_composite(pil_image.convert('RGBA'), overlay).convert('RGB')
            
            return np.array(pil_image)
            
        except Exception as e:
            print(f"❌ 색상 박스 생성 실패: {e}")
            # 빈 이미지 반환
            return np.ones((300, 300, 3), dtype=np.uint8) * 255
    
    def _create_line_statistics(self, vertical_lines: List[List[Dict]]) -> Dict:
        """세로줄별 상세 통계 생성"""
        line_stats = {}
        
        for line_idx, line_words in enumerate(vertical_lines):
            confidences = [word['confidence'] for word in line_words]
            texts = [word['content'] for word in line_words]
            
            # 바운딩 박스 계산
            all_points = []
            for word in line_words:
                polygon = word['polygon']
                points = [(polygon[i], polygon[i+1]) for i in range(0, len(polygon), 2)]
                all_points.extend(points)
            
            if all_points:
                min_x = min(point[0] for point in all_points)
                max_x = max(point[0] for point in all_points)
                min_y = min(point[1] for point in all_points)
                max_y = max(point[1] for point in all_points)
                
                padding = 10
                bbox = (
                    int(min_x - padding),
                    int(min_y - padding),
                    int(max_x - min_x + 2 * padding),
                    int(max_y - min_y + 2 * padding)
                )
            else:
                bbox = (0, 0, 0, 0)
            
            # 개별 단어 바운딩 박스
            word_bboxes = []
            for word in line_words:
                polygon = word['polygon']
                points = [(polygon[i], polygon[i+1]) for i in range(0, len(polygon), 2)]
                
                if points:
                    min_x = min(point[0] for point in points)
                    max_x = max(point[0] for point in points)
                    min_y = min(point[1] for point in points)
                    max_y = max(point[1] for point in points)
                    
                    word_bboxes.append({
                        'text': word['content'],
                        'bbox': (int(min_x), int(min_y), int(max_x - min_x), int(max_y - min_y)),
                        'confidence': word['confidence']
                    })
            
            line_stats[f'Line {line_idx + 1}'] = {
                'bbox': bbox,
                'words': word_bboxes,
                'word_count': len(line_words),
                'avg_confidence': np.mean(confidences) if confidences else 0,
                'full_text': ''.join(texts)
            }
        
        return line_stats
    
    def analyze_document_complete(self, image_path: str, save_visualization: bool = True) -> Optional[Dict]:
        """완전한 문서 분석 (Azure API → 시각화)"""
        print(f"🚀 완전한 Azure OCR 분석 시작")
        print(f"📄 대상 파일: {os.path.basename(image_path)}")
        
        # 1. Azure로 실시간 분석
        ocr_data = self.analyze_document_live(image_path)
        
        if not ocr_data:
            print("❌ OCR 분석 실패")
            return None
        
        # 2. 시각화 생성
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        visualization_path = None
        
        if save_visualization:
            visualization_path = f"{Path(image_path).stem}_visualization_{timestamp}.png"
        
        line_stats = self.create_dual_visualization(image_path, ocr_data, visualization_path)
        
        print("✅ 완전한 분석 완료!")
        return line_stats


def create_azure_config():
    """Azure 설정 파일 자동 생성 (환경변수 우선 권장)"""
    print("📝 Azure 설정 방법:")
    print("=" * 50)
    print("권장: .env 파일 사용 (보안상 안전)")
    print("1. 프로젝트 루트에 .env 파일 생성")
    print("2. 다음 내용 추가:")
    print("   AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT=https://your-resource.cognitiveservices.azure.com/")
    print("   AZURE_DOCUMENT_INTELLIGENCE_KEY=your-api-key-here")
    print("\n대안: config.json 파일 생성")
    
    config_template = {
        "endpoint": "https://your-resource-name.cognitiveservices.azure.com/",
        "key": "your-32-character-api-key-here",
        "description": "Azure Document Intelligence 설정 파일",
        "created": datetime.now().isoformat(),
        "instructions": [
            "⚠️  보안상 .env 파일 사용을 권장합니다",
            "1. Azure Portal에서 Document Intelligence 리소스 생성",
            "2. 키 및 엔드포인트 정보를 위 필드에 입력",
            "3. 파일을 config.json으로 저장",
            "4. .gitignore에 config.json 추가하여 보안 유지"
        ]
    }
    
    config_path = "config.json"
    
    try:
        with open(config_path, 'w', encoding='utf-8') as f:
            json.dump(config_template, f, indent=2, ensure_ascii=False)
        
        print(f"✅ Azure 설정 템플릿 생성: {config_path}")
        print("⚠️  보안을 위해 .env 파일 사용을 권장합니다!")
        
        return config_path
        
    except Exception as e:
        print(f"❌ 설정 파일 생성 실패: {e}")
        return None


def main():
    """메인 실행 함수 - 환경변수 기반 실행"""
    print("🚀 Azure Document Intelligence 완전 OCR 분석기")
    print("=" * 60)
    
    # 환경변수 확인
    endpoint = os.getenv('AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT')
    key = os.getenv('AZURE_DOCUMENT_INTELLIGENCE_KEY')
    
    if not endpoint or not key:
        print("⚠️ Azure 환경변수가 설정되지 않았습니다.")
        print("💡 .env 파일에 Azure 설정을 추가하거나 create_azure_config()를 실행하세요.")
        create_azure_config()
        return
    
    # OCR 분석기 초기화
    try:
        analyzer = CompleteAzureOCRAnalyzer()
    except Exception as e:
        print(f"❌ 분석기 초기화 실패: {e}")
        return
    
    # 분석할 이미지 경로 (환경변수로 설정 가능)
    default_image_path = os.getenv('DEFAULT_IMAGE_PATH', 
                                  r"C:\Users\EL76\OneDrive\Desktop\IT\Project\5eyes-Hoyeon\data\raw\training\원천\태조실록_001권_총서_001a면.jpg")
    
    image_paths = [default_image_path]
    
    # 존재하는 첫 번째 이미지 파일 찾기
    target_image = None
    for img_path in image_paths:
        if os.path.exists(img_path):
            target_image = img_path
            break
    
    if target_image:
        print(f"\n📄 분석 대상: {target_image}")
        
        # 완전한 분석 실행
        result = analyzer.analyze_document_complete(
            image_path=target_image,
            save_visualization=True
        )
        
        if result:
            print("\n🎉 분석 성공! 결과가 저장되었습니다.")
        else:
            print("\n❌ 분석 실패")
    
    else:
        print("\n❌ 분석할 이미지 파일을 찾을 수 없습니다.")
        print(f"📝 다음 경로에 파일을 준비하세요: {default_image_path}")
        print("🔧 또는 DEFAULT_IMAGE_PATH 환경변수를 설정하세요.")


if __name__ == "__main__":
    main()